In [302]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [303]:
#Loading the datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
weather = pd.read_csv('weather.csv')
spray = pd.read_csv('spray.csv')

WEATHER

Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level                
Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

In [304]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [305]:
weather.shape

(2944, 22)

In [306]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Station      2944 non-null   int64  
 1   Date         2944 non-null   object 
 2   Tmax         2944 non-null   int64  
 3   Tmin         2944 non-null   int64  
 4   Tavg         2944 non-null   object 
 5   Depart       2944 non-null   object 
 6   DewPoint     2944 non-null   int64  
 7   WetBulb      2944 non-null   object 
 8   Heat         2944 non-null   object 
 9   Cool         2944 non-null   object 
 10  Sunrise      2944 non-null   object 
 11  Sunset       2944 non-null   object 
 12  CodeSum      2944 non-null   object 
 13  Depth        2944 non-null   object 
 14  Water1       2944 non-null   object 
 15  SnowFall     2944 non-null   object 
 16  PrecipTotal  2944 non-null   object 
 17  StnPressure  2944 non-null   object 
 18  SeaLevel     2944 non-null   object 
 19  Result

In [307]:
weather.describe()

,Station,Tmax,Tmin,DewPoint,ResultSpeed,ResultDir
count,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000
mean,1.500000,76.166101,57.810462,53.457880,6.960666,17.494905
std,0.500085,11.461970,10.381939,10.675181,3.587527,10.063609
min,1.000000,41.000000,29.000000,22.000000,0.100000,1.000000
25%,1.000000,69.000000,50.000000,46.000000,4.300000,7.000000
50%,1.500000,78.000000,59.000000,54.000000,6.400000,19.000000
75%,2.000000,85.000000,66.000000,62.000000,9.200000,25.000000
max,2.000000,104.000000,83.000000,75.000000,24.100000,36.000000


In [308]:
#Check for missing data
weather.isnull().sum()

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0
Water1         0
SnowFall       0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

In [309]:
#Check for missing data M
weather.isin(['M']).sum()[weather.isin(['M']).sum()>0].sort_values(ascending = False )

Water1         2944
Depart         1472
Depth          1472
SnowFall       1472
Tavg             11
Heat             11
Cool             11
SeaLevel          9
WetBulb           4
StnPressure       4
AvgSpeed          3
PrecipTotal       2
dtype: int64

In [310]:
#Check for duplicate data
weather.duplicated().sum()

0

In [311]:
#Check for Trace values
weather.isin(['  T']).sum()[weather.isin(['  T']).sum()>0].sort_values(ascending = False)

PrecipTotal    318
SnowFall        12
dtype: int64

In [312]:
#Check for zero (0) values
weather.isin(['0']).sum()[weather.isin(['0']).sum()>0].sort_values(ascending = False )

Heat     1870
Depth    1472
dtype: int64

In [313]:
#Check for zero (0.0) values
weather.isin(['0.0']).sum()[weather.isin(['0.0']).sum()>0].sort_values(ascending = False )

SnowFall    1459
dtype: int64

In [314]:
#Convert date from object to datetime format
weather['Date'] = pd.to_datetime(weather['Date'])

IMPUTING AND DROPING FEATURES

In [315]:
#Droping columns with high amount of missing data
weather = weather.drop(columns = ['Water1', 'Depth', 'SnowFall'])

In [316]:
#Imputing missing data for Tavg

def impute_Tavg(row):
    if row['Tavg'] == 'M':
        row['Tavg'] = round((row['Tmax'] + row['Tmin']) / 2)
    return row   

In [317]:
#Imputing missing data for Heat and Cool

def impute_Heat_Cool(row):
    
    if row['Cool'] == 'M' or row['Heat'] == 'M':
        t = 65 - row['Tavg']
        if t < 0: 
            row['Cool'] = t
            row['Heat'] = 0
        elif t > 0:
            row['Cool'] = 0
            row['Heat'] = t
        else:
            row['Cool'] = row['Heat'] = 0
    return row

In [318]:
#Apply the function impute_Tavg and impute_Heat_Cool
weather = weather.apply(impute_Tavg, axis = 1)
weather = weather.apply (impute_Heat_Cool, axis = 1)

In [319]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0.00,29.39,30.12,11.7,7,11.9


In [320]:
#Function to extract 30 year normal temperature from station 1's readings and apply it to station 2's readings

def create_Norm(row):
    if row['Station'] == 1:
        row['Normal'] = int(row['Tavg']) - int(row['Depart'])
    return row


def apply_Norm(row):
    if row['Station'] == 2:
        row['Normal'] = weather[(weather['Date'] == row['Date']) & (weather['Station'] == 1)]['Normal'].values[0]
    return row

In [321]:
#Apply the function create_Norm and apply_Norm
weather = weather.apply(create_Norm, axis=1)
weather = weather.apply(apply_Norm, axis=1)

In [322]:
#Rearrange columns
weather = weather[['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Normal', 'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 
                   'Sunrise', 'Sunset', 'CodeSum','PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir',
                   'AvgSpeed']]

In [323]:
#Remaining 'M' values
weather.isin(['M']).sum()[weather.isin(['M']).sum() > 0]

Depart         1472
WetBulb           4
PrecipTotal       2
StnPressure       4
SeaLevel          9
AvgSpeed          3
dtype: int64

In [324]:
#Impute remaining by retrieving from Station 1 if Station 2 is 'M' and vice versa

def impute_remaining(row):
    replace_dict = {}
    for index in row.index:
        if row[index] == 'M':
            replace_dict[index] = 'M'
    if replace_dict:
        if 'Depart' in replace_dict:
            row['Depart'] = int(row['Tavg']) - int(row['Normal'])
            del replace_dict['Depart']
            
        for key, value in replace_dict.items():
            if row['Station'] == 2:
                row[key] = weather[(weather['Date'] == row['Date']) & (weather['Station'] == 1)][key].values[0]
            else:
                row[key] = weather[(weather['Date'] == row['Date']) & (weather['Station'] == 2)][key].values[0]                
    return row

In [325]:
#Apply the function impute_remaining
weather = weather.apply(impute_remaining, axis=1)

In [326]:
#Impute according to the StnPressure of the day after for Station 1 and 2 = 'M'

weather.at[2410, 'StnPressure'] = weather[weather['Date'] == '2013-08-11']['StnPressure'].values[0]
weather.at[2411, 'StnPressure'] = weather[weather['Date'] == '2013-08-11']['StnPressure'].values[1]

In [329]:
#Setting the Trace to 0.01
weather['PrecipTotal'] = weather['PrecipTotal'].apply(lambda x: 0.01 if 'T' in x else x)

In [330]:
#Impute sunrise/sunset

def impute_sun(row):
    if row['Station'] == 2:
        row['Sunrise'] = weather[(weather['Date'] == row['Date']) & (weather['Station'] == 1)]['Sunrise'].values[0]
        row['Sunset'] = weather[(weather['Date'] == row['Date']) & (weather['Station'] == 1)]['Sunset'].values[0]
    return row

In [331]:
#Apply the function impute_sun
weather = weather.apply(impute_sun, axis=1)

In [332]:
#Uniwur weather conditions 
weather['CodeSum'].unique()

array([' ', 'BR', 'BR HZ', 'HZ', 'RA', 'RA BR', 'TSRA RA BR', 'RA VCTS',
       'TSRA RA', 'RA HZ', 'TSRA RA BR HZ', 'TSRA BR HZ', 'RA BR HZ VCTS',
       'TSRA RA HZ', 'TSRA BR HZ VCTS', 'TSRA', 'TSRA BR HZ FU',
       'TSRA RA HZ FU', 'BR HZ FU', 'TSRA RA VCTS', 'HZ VCTS', 'TSRA HZ',
       'VCTS', 'RA BR VCTS', 'TSRA RA BR VCTS', 'TS TSRA RA BR HZ VCTS',
       'DZ BR', 'TS TSRA RA BR HZ', 'TS TSRA BR HZ', 'RA BR HZ',
       'TSRA RA DZ BR HZ', 'TS TSRA RA BR', 'TS RA BR', 'TS TSRA RA',
       'TS TSRA RA BR VCTS', 'TS TSRA BR', 'TS RA', 'RA BCFG BR',
       'TSRA BR', 'RA DZ FG+ BCFG BR', 'RA FG+ MIFG BR', 'RA DZ',
       'RA DZ BR', 'TS TSRA RA HZ', 'TSRA RA FG+ FG BR',
       'TSRA DZ FG+ FG BR HZ', 'TS BR', 'RA BR SQ', 'TS TSRA',
       'TSRA RA BR HZ VCTS', 'BR VCTS', 'TS', 'FG+ BR HZ', 'RA SN',
       'TSRA RA DZ BR', 'DZ BR HZ', 'RA BR FU', 'TS BR HZ', 'DZ',
       'FG+ BR', 'FG+ FG BR', 'FG+ MIFG BR', 'TSRA RA FG BR',
       'TSRA FG+ BR', 'RA DZ BR HZ', 'RA DZ SN', 'FG+ FG 

In [333]:
#Ensuring that each code has proper spacing
codes = ['+FC','FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL',
         'IC', 'FG+', 'FG', 'BR', 'UP', 'HZ', 'FU', 'VA', 'DU', 'DS',
         'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR',
         'BC', 'BL', 'VC']
weather['CodeSum'] = weather['CodeSum'].apply(lambda x: ' '.join([t for t in x.split(' ') if t in codes]))

In [334]:
#Measure the number of unique weather conditions each day
weather['n_codesum'] = weather['CodeSum'].apply(lambda x: len(x.split()))

In [335]:
#Creating variables Rain, Mist, Snow, Wind instead of CodeSum

weather["Rain"] = np.where(weather['CodeSum'].str.contains("|".join(['TS', 'GR', 'RA', 'DZ', 'SH'])), 1, 0)
weather["Mist"] = np.where(weather['CodeSum'].str.contains("|".join(['FG+', 'FG', 'BR', 'HZ'])), 1, 0)
weather["Snow"] = np.where(weather['CodeSum'].str.contains("|".join(['SN', 'SG', 'GS', 'PL', 'IC', 'DR', 'BC'])), 1, 0)
weather["Wind"] = np.where(weather['CodeSum'].str.contains("|".join(['SQ', 'DS', 'SS', 'PO', 'BL'])), 1, 0)

In [336]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Normal,Depart,DewPoint,WetBulb,Heat,...,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,n_codesum,Rain,Mist,Snow,Wind
0,1,2007-05-01,83,50,67,53.0,14,51,56,0,...,29.10,29.82,1.7,27,9.2,0,0,0,0,0
1,2,2007-05-01,84,52,68,53.0,15,51,57,0,...,29.18,29.82,2.7,25,9.6,0,0,0,0,0
2,1,2007-05-02,59,42,51,54.0,-3,42,47,14,...,29.38,30.09,13.0,4,13.4,1,0,1,0,0
3,2,2007-05-02,60,43,52,54.0,-2,42,47,13,...,29.44,30.08,13.3,2,13.4,2,0,1,0,0
4,1,2007-05-03,66,46,56,54.0,2,40,48,9,...,29.39,30.12,11.7,7,11.9,0,0,0,0,0


In [337]:
#Count of Rain, Mist, Snow, Wind
weather[['Rain', 'Mist', 'Snow', 'Wind']].sum()

Rain    1007
Mist     882
Snow       6
Wind       3
dtype: int64

In [338]:
#Dropping columns Snow and Wind because of low values
weather = weather.drop(columns = ['Snow', 'Wind']) 

In [339]:
#Convert remaining columns

for col in weather.columns:
    try:
        weather[col] = weather[col].astype(float)
    except:
        print(col, 'cannot convert to float')
        pass

Date cannot convert to float
CodeSum cannot convert to float


In [340]:
#Checking the dataset
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Station      2944 non-null   float64       
 1   Date         2944 non-null   datetime64[ns]
 2   Tmax         2944 non-null   float64       
 3   Tmin         2944 non-null   float64       
 4   Tavg         2944 non-null   float64       
 5   Normal       2944 non-null   float64       
 6   Depart       2944 non-null   float64       
 7   DewPoint     2944 non-null   float64       
 8   WetBulb      2944 non-null   float64       
 9   Heat         2944 non-null   float64       
 10  Cool         2944 non-null   float64       
 11  Sunrise      2944 non-null   float64       
 12  Sunset       2944 non-null   float64       
 13  CodeSum      2944 non-null   object        
 14  PrecipTotal  2944 non-null   float64       
 15  StnPressure  2944 non-null   float64       
 16  SeaLev

In [341]:
#Merge both Station 1 and Station 2
weather = weather.groupby('Date').sum() / 2
weather = weather.drop(columns=['Station']).reset_index()

In [342]:
#Drop column Normal since no longer required
weather = weather.drop(columns='Normal')

In [343]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         1472 non-null   datetime64[ns]
 1   Tmax         1472 non-null   float64       
 2   Tmin         1472 non-null   float64       
 3   Tavg         1472 non-null   float64       
 4   Depart       1472 non-null   float64       
 5   DewPoint     1472 non-null   float64       
 6   WetBulb      1472 non-null   float64       
 7   Heat         1472 non-null   float64       
 8   Cool         1472 non-null   float64       
 9   Sunrise      1472 non-null   float64       
 10  Sunset       1472 non-null   float64       
 11  PrecipTotal  1472 non-null   float64       
 12  StnPressure  1472 non-null   float64       
 13  SeaLevel     1472 non-null   float64       
 14  ResultSpeed  1472 non-null   float64       
 15  ResultDir    1472 non-null   float64       
 16  AvgSpe

In [344]:
weather

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,n_codesum,Rain,Mist
0,2007-05-01,83.5,51.0,67.5,14.5,51.0,56.5,0.0,2.5,448.0,1849.0,0.000,29.140,29.820,2.20,26.0,9.40,0.0,0.0,0.0
1,2007-05-02,59.5,42.5,51.5,-2.5,42.0,47.0,13.5,0.0,447.0,1850.0,0.000,29.410,30.085,13.15,3.0,13.40,1.5,0.0,1.0
2,2007-05-03,66.5,47.0,57.0,3.0,40.0,49.0,8.0,0.0,446.0,1851.0,0.000,29.425,30.120,12.30,6.5,12.55,0.5,0.0,0.5
3,2007-05-04,72.0,50.0,61.0,7.0,41.5,50.0,4.0,0.0,444.0,1852.0,0.005,29.335,30.045,10.25,7.5,10.60,0.5,0.5,0.0
4,2007-05-05,66.0,53.5,60.0,5.0,38.5,49.5,5.0,0.0,443.0,1853.0,0.010,29.430,30.095,11.45,7.0,11.75,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,2014-10-27,78.0,52.5,65.5,17.5,51.5,58.5,0.5,1.0,618.0,1653.0,0.010,28.960,29.665,12.35,19.0,13.25,0.5,0.5,0.0
1468,2014-10-28,67.0,46.5,57.0,10.0,39.0,47.5,8.0,0.0,619.0,1651.0,0.020,29.190,29.850,14.40,26.0,15.10,0.5,0.5,0.0
1469,2014-10-29,49.0,38.0,44.0,-3.0,33.0,41.0,21.0,0.0,620.0,1650.0,0.000,29.390,30.065,9.00,29.0,9.45,0.0,0.0,0.0
1470,2014-10-30,52.0,34.5,43.5,-2.5,34.5,41.0,21.5,0.0,622.0,1649.0,0.005,29.375,30.095,5.50,23.5,6.00,0.5,0.5,0.0


In [345]:
#Add Year, Month, Week and Day of Week features

weather['Year'] = weather['Date'].apply(lambda x: x.year)
weather['Month'] = weather['Date'].apply(lambda x: x.month)
weather['Week'] = weather['Date'].apply(lambda x: x.week)
weather['DayOfWeek'] = weather['Date'].apply(lambda x: x.dayofweek)

In [346]:
weather.to_csv("Cleaned_weather.csv", index = False)

SPRAY

In [347]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [348]:
spray.shape

(14835, 4)

In [349]:
spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       14835 non-null  object 
 1   Time       14251 non-null  object 
 2   Latitude   14835 non-null  float64
 3   Longitude  14835 non-null  float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [350]:
spray.describe()

,Latitude,Longitude
count,14835.000000,14835.000000
mean,41.904828,-87.736690
std,0.104381,0.067292
min,41.713925,-88.096468
25%,41.785001,-87.794225
50%,41.940075,-87.727853
75%,41.980978,-87.694108
max,42.395983,-87.586727


In [351]:
#check for missing data
spray.isnull().sum()

Date           0
Time         584
Latitude       0
Longitude      0
dtype: int64

In [352]:
spray.Time.value_counts().head(10)

7:44:32 PM    541
9:05:56 PM      5
8:58:56 PM      5
8:55:46 PM      5
8:58:26 PM      5
9:35:47 PM      5
9:40:27 PM      5
8:55:26 PM      5
9:38:27 PM      5
8:59:06 PM      5
Name: Time, dtype: int64

In [353]:
spray.loc[spray.Time == '7:44:32 PM']

,Date,Time,Latitude,Longitude
489,2011-09-07,7:44:32 PM,41.98646,-87.794225
490,2011-09-07,7:44:32 PM,41.98646,-87.794225
491,2011-09-07,7:44:32 PM,41.98646,-87.794225
492,2011-09-07,7:44:32 PM,41.98646,-87.794225
493,2011-09-07,7:44:32 PM,41.98646,-87.794225
...,...,...,...,...
1025,2011-09-07,7:44:32 PM,41.98646,-87.794225
1026,2011-09-07,7:44:32 PM,41.98646,-87.794225
1027,2011-09-07,7:44:32 PM,41.98646,-87.794225
1028,2011-09-07,7:44:32 PM,41.98646,-87.794225


In [354]:
spray.loc[spray.Time == '9:05:56 PM']

,Date,Time,Latitude,Longitude
2027,2011-09-07,9:05:56 PM,41.977213,-87.808097
4469,2013-07-25,9:05:56 PM,41.952365,-87.735402
5709,2013-07-25,9:05:56 PM,41.963438,-87.726603
8382,2013-08-15,9:05:56 PM,41.900408,-87.763738
13798,2013-08-29,9:05:56 PM,41.983538,-87.680232


In [355]:
spray.loc[spray.Time == '8:58:56 PM']

,Date,Time,Latitude,Longitude
1988,2011-09-07,8:58:56 PM,41.975408,-87.814512
4445,2013-07-25,8:58:56 PM,41.952233,-87.736818
5684,2013-07-25,8:58:56 PM,41.965338,-87.729188
8355,2013-08-15,8:58:56 PM,41.907147,-87.767105
13764,2013-08-29,8:58:56 PM,41.985008,-87.682222


In [356]:
#Drop the duplicates
spray.drop_duplicates(inplace=True)

In [357]:
#Drop the column Time since it has many missing values and can't find a pattern to impute
spray.drop(columns='Time',inplace=True)

In [358]:
#Convert date from object to datetime format
spray['Date'] = pd.to_datetime(spray['Date'])

In [359]:
spray.head()

,Date,Latitude,Longitude
0,2011-08-29,42.391623,-88.089163
1,2011-08-29,42.391348,-88.089163
2,2011-08-29,42.391022,-88.089157
3,2011-08-29,42.390637,-88.089158
4,2011-08-29,42.390410,-88.088858


In [360]:
spray.to_csv("Cleaned_spray.csv", index = False)

TRAIN

In [361]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [362]:
#Add Year, Month, Week and Day of Week features
train['Date'] = pd.to_datetime(train['Date'])
train['Year'] = train['Date'].apply(lambda x: x.year)
train['Month'] = train['Date'].apply(lambda x: x.month)
train['Week'] = train['Date'].apply(lambda x: x.week)
train['DayOfWeek'] = train['Date'].apply(lambda x: x.dayofweek)


In [363]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    10506 non-null  datetime64[ns]
 1   Address                 10506 non-null  object        
 2   Species                 10506 non-null  object        
 3   Block                   10506 non-null  int64         
 4   Street                  10506 non-null  object        
 5   Trap                    10506 non-null  object        
 6   AddressNumberAndStreet  10506 non-null  object        
 7   Latitude                10506 non-null  float64       
 8   Longitude               10506 non-null  float64       
 9   AddressAccuracy         10506 non-null  int64         
 10  NumMosquitos            10506 non-null  int64         
 11  WnvPresent              10506 non-null  int64         
 12  Year                    10506 non-null  int64 

In [364]:
train.shape

(10506, 16)

In [365]:
train.describe()

,Block,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Year,Month,Week,DayOfWeek
count,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000
mean,35.687797,41.841139,-87.699908,7.819532,12.853512,0.052446,2009.576242,7.692557,31.687131,2.610889
std,24.339468,0.112742,0.096514,1.452921,16.133816,0.222936,2.354645,1.067675,4.526874,1.380745
min,10.000000,41.644612,-87.930995,3.000000,1.000000,0.000000,2007.000000,5.000000,22.000000,0.000000
25%,12.000000,41.732984,-87.760070,8.000000,2.000000,0.000000,2007.000000,7.000000,28.000000,2.000000
50%,33.000000,41.846283,-87.694991,8.000000,5.000000,0.000000,2009.000000,8.000000,31.000000,3.000000
75%,52.000000,41.954690,-87.627796,9.000000,17.000000,0.000000,2011.000000,8.000000,35.000000,4.000000
max,98.000000,42.017430,-87.531635,9.000000,50.000000,1.000000,2013.000000,10.000000,41.000000,4.000000


In [366]:
train['WnvPresent'] = train['WnvPresent'].map(lambda x : 1 if x > 0 else x)

In [367]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Year,Month,Week,DayOfWeek
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,2007,5,22,1
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,2007,5,22,1
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,2007,5,22,1
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,2007,5,22,1
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,2007,5,22,1


In [368]:
train.to_csv("Cleaned_train.csv", index = False)

TEST

In [369]:
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,Year,Month,Week,DayOfWeek
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,2008,6,24,2
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,2008,6,24,2
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,2008,6,24,2
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,2008,6,24,2
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,2008,6,24,2


In [370]:
test.shape

(116293, 15)

In [371]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Id                      116293 non-null  int64  
 1   Date                    116293 non-null  object 
 2   Address                 116293 non-null  object 
 3   Species                 116293 non-null  object 
 4   Block                   116293 non-null  int64  
 5   Street                  116293 non-null  object 
 6   Trap                    116293 non-null  object 
 7   AddressNumberAndStreet  116293 non-null  object 
 8   Latitude                116293 non-null  float64
 9   Longitude               116293 non-null  float64
 10  AddressAccuracy         116293 non-null  int64  
 11  Year                    116293 non-null  int64  
 12  Month                   116293 non-null  int64  
 13  Week                    116293 non-null  int64  
 14  DayOfWeek           

In [372]:
test.describe()

,Id,Block,Latitude,Longitude,AddressAccuracy,Year,Month,Week,DayOfWeek
count,116293.000000,116293.000000,116293.000000,116293.000000,116293.000000,116293.000000,116293.000000,116293.000000,116293.000000
mean,58147.000000,41.131100,41.849389,-87.693658,7.954357,2010.702759,7.620244,31.417833,2.598978
std,33571.041765,24.864726,0.106593,0.080699,1.252733,2.128750,1.065842,4.660725,1.418081
min,1.000000,10.000000,41.644612,-87.930995,3.000000,2008.000000,6.000000,22.000000,0.000000
25%,29074.000000,18.000000,41.753411,-87.750938,8.000000,2008.000000,7.000000,28.000000,1.000000
50%,58147.000000,39.000000,41.862292,-87.694991,8.000000,2010.000000,8.000000,31.000000,3.000000
75%,87220.000000,61.000000,41.951866,-87.648860,9.000000,2012.000000,8.000000,35.000000,4.000000
max,116293.000000,98.000000,42.017430,-87.531635,9.000000,2014.000000,10.000000,40.000000,4.000000


In [373]:
#Add Year, Month, Week and Day of Week features
test['Date'] = pd.to_datetime(test['Date'])
test['Year'] = test['Date'].apply(lambda x: x.year)
test['Month'] = test['Date'].apply(lambda x: x.month)
test['Week'] = test['Date'].apply(lambda x: x.week)
test['DayOfWeek'] = test['Date'].apply(lambda x: x.dayofweek)


In [374]:
test.to_csv("test.csv", index = False)